<a href="https://colab.research.google.com/github/nurfnick/Mathematical_Musings/blob/main/GradDescent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Gradient Descent for Training Linear Regression

In multiple linear regression, the goal is to minimize the sum of the squared errors.

$$ RSS = \sum_{i=1}^n \left(y_i-\hat y_i\right)^2 = (\vec y - X \vec \beta)^T(\vec y - X\vec \beta) $$

This first equation is in the traditional format (where $\hat y$ is the prediction $\vec x \cdot \vec \beta$) and the second in a matrix format.

We know the solution to this problem analytically, $\hat \beta = \left(X^TX\right)^{-1}X^T\vec y$.  What I am interested in is if I can code to find an approximation of this solution by following the gradient descent.

The gradient descent is an iterative process where by we initialize a solution (for $\beta$) and update it by using $\beta - w \nabla RSS$ where $w$ is some user chosen weight and 
$$
\nabla RSS = -2X^T\left(\vec y - X\vec \beta\right)
$$

Okay enough with the theory, let's get our hands dirty with some coding!  I'll start with some data and show the solution for $\hat \beta$.

In [2]:
import numpy as np
import pandas as pa
from sklearn.linear_model import LinearRegression

df = pa.read_csv('https://raw.githubusercontent.com/nurfnick/Applied_Stats_Jupyter_Notebooks/master/blues.csv')

In [3]:
df.head()

,Rk,Player,From,To,Yrs,Lg,GP,G,A,PTS,+/-,PIM,EV,PP,SH,GW,EV.1,PP.1,SH.1,S,S%,TOI,ATOI
0,1,Bruce Affleck\afflebr01,1975,1979,5,NHL,274,14,65,79,-81,86,10,4,0,2,48,14,3,363,3.9,NaN,NaN
1,2,Kenny Agostino\agostke01,2017,2017,1,NHL,7,1,2,3,0,2,1,0,0,0,1,1,0,17,5.9,89.0,12:47
2,3,Glenn Anderson*\andergl01,1995,1996,2,NHL,51,14,16,30,-2,43,12,2,0,3,13,3,0,89,15.7,NaN,NaN
3,4,Perry Anderson\anderpe01,1982,1985,4,NHL,144,22,18,40,-14,355,22,0,0,2,18,0,0,168,13.1,NaN,NaN
4,5,Ron Anderson\anderro01,1970,1970,1,NHL,59,9,9,18,9,36,8,1,0,0,8,1,0,107,8.4,NaN,NaN


This dataset is (most?) of the players for my favorite hockey team.  Let's try to predict **GP** from **G**, **A**, and **+/-**.  If you aren't familiar with these you can gain some content knowledge about hockey [here](https://en.wikipedia.org/wiki/Ice_hockey)

In [4]:
X= np.array(df[['G', 'A', '+/-']])
y = np.array(df['GP'])


In [5]:
X = np.insert(X,3,1,axis = 1)#Adding 1's to get the intercept too!

By the above equation...

In [6]:
XX = np.dot(X.transpose(),X)
z = np.linalg.inv(XX)
w = z@X.transpose()
np.dot(w,y)

array([ 0.32673324,  1.79968341,  0.21442554, 50.11143093])

Let's double check with a built-in call.

In [7]:
from sklearn.linear_model import LinearRegression
linreg = LinearRegression()#define it as a regular linear regression
linreg.fit(X,y)#This fits it!
linreg.coef_

array([0.32673324, 1.79968341, 0.21442554, 0.        ])

The final 0 is just because it casts the intercept differently then we have, it is displayed next.

In [8]:
linreg.intercept_

50.11143092932841

So we know the answer, great, let's try to estimate it!

In [46]:
b = np.random.normal(0,5,4)#intitialize
w = 0.0001 #a weight for the descent
def grad(b):
  return -2/len(y)*(X.transpose())@(y-X@b)

for i in range(100000):
  b = b - w*grad(b)
  
b

array([ 0.32673322,  1.79968351,  0.2144255 , 50.11141972])

This is working only after much consternation.  I really thought I would see convergence quickly!  Let's revisit this code and show how it is doing in a few key places

In [32]:
b = np.random.normal(0,5,4)#intitialize
w = 0.0001 #a weight for the descent

for i in range(50000):
  b = b - w*grad(b)
  if i % 1000 == 0:
    print(b)

[1.23413989 2.56654757 6.24444646 6.65501249]
[ 0.23842688  2.11006871  0.08644685 12.80332648]
[ 0.25088538  2.06627869  0.10450244 18.06685743]
[ 0.26158621  2.02866669  0.1200107  22.58779483]
[ 0.27077734  1.99636109  0.133331   26.47090583]
[ 0.27867175  1.96861326  0.14477205 29.80617675]
[ 0.2854524   1.94478017  0.15459895 32.67089838]
[ 0.29127642  1.92430952  0.16303945 35.13145714]
[ 0.29627876  1.90672692  0.17028915 37.24487343]
[ 0.30057537  1.89162492  0.17651603 39.06012308]
[ 0.30426579  1.87865356  0.18186441 40.61927228]
[ 0.30743556  1.86751223  0.18645822 41.95845237]
[ 0.31015813  1.85794275  0.19040393 43.10869722]
[ 0.3124966   1.84972337  0.19379296 44.09666234]
[ 0.31450514  1.84266359  0.19670386 44.94524259]
[ 0.31623032  1.83659983  0.19920409 45.67410278]
[ 0.3177121   1.83139156  0.20135157 46.30013336]
[ 0.31898483  1.82691808  0.20319608 46.8378418 ]
[ 0.32007799  1.82307574  0.20478037 47.29968884]
[ 0.32101694  1.81977549  0.20614113 47.69637722]
[ 0.

After 1000 iterations it isn't even close, like it may never converge but by 10 000 it starts to look like the values are only changing a little.  I think this is part of the problem with gradient descent is that it just goes really slowly as you get close!

Maybe adding in some noise will help?

In [51]:
b = np.random.normal(0,5,4)#intitialize
w = 0.001 #a weight for the descent

for i in range(100000):
  b = b - w*np.random.normal(0,1)*grad(b)/(i+1)**2
  
print(b)

[-0.94595788  0.0774263  -0.34388532  0.52931364]


Doesn't seem too.  I added a feature to diminish the noise as the process has been running longer.